In [154]:
from sympy import Eq, solve, pi
from sympy import lambdify
import numpy as np
from scipy.optimize import fsolve


from sympy import Matrix, cos, sin, symbols

def rotation_matrix_2d(theta):
    """Generate a 2D rotation matrix for a given angle."""
    return Matrix([
        [cos(theta), -sin(theta)],
        [sin(theta), cos(theta)]
    ])

def generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j):
    """Generate the position restraint equations."""
    R_i = Matrix([r_x_i, r_y_i])
    R_j = Matrix([r_x_j, r_y_j])
    local_vector_i = Matrix([u_x_i, u_y_i])
    local_vector_j = Matrix([u_x_j, u_y_j])
    
    transformed_vector_i = rotation_matrix_2d(theta_i) * local_vector_i
    transformed_vector_j = rotation_matrix_2d(theta_j) * local_vector_j

    constraint_equations = R_i + transformed_vector_i - R_j - transformed_vector_j
    return constraint_equations

def generate_equations_wo_topology_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j):

    u_x_i, u_y_i, u_x_j, u_y_j = symbols('u_x_i u_y_i u_x_j u_y_j')


    # Generate the constraint equations
    equations = generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j)    
    
    # Return the equation and the symbols
    #return (equations)
    return equations

def generate_equations_included_topology_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_a, u_y_a, u_x_b, u_y_b):

    u_x_i, u_y_i, u_x_j, u_y_j = symbols('u_x_i u_y_i u_x_j u_y_j')


    # Generate the constraint equations
    equations = generate_constraint_equations_pr(theta_i, theta_j, r_x_i, r_y_i, r_x_j, r_y_j, u_x_i, u_y_i, u_x_j, u_y_j)
    
    # Substitute the symbolic variables with the provided values
    equations_substituted = equations.subs({u_x_i: u_x_a, u_y_i: u_y_a, u_x_j: u_x_b, u_y_j: u_y_b})
    
    
    # Return the equation and the symbols
    #return (equations)
    return equations_substituted


In [155]:
theta_1, theta_2 = symbols('theta_1 theta_2')
r_x_1, r_y_1, r_x_2, r_y_2 = symbols('r_x_1 r_y_1 r_x_2 r_y_2')

generate_equations_wo_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2)

Matrix([
[r_x_1 - r_x_2 + u_x_i*cos(theta_1) - u_x_j*cos(theta_2) - u_y_i*sin(theta_1) + u_y_j*sin(theta_2)],
[r_y_1 - r_y_2 + u_x_i*sin(theta_1) - u_x_j*sin(theta_2) + u_y_i*cos(theta_1) - u_y_j*cos(theta_2)]])

In [156]:


theta_1, theta_2 = symbols('theta_1 theta_2')
r_x_1, r_y_1, r_x_2, r_y_2 = symbols('r_x_1 r_y_1 r_x_2 r_y_2')


u_x_1_12 = 0
u_y_1_12 = 0
u_x_2_12 = 0
u_y_2_12 = 0
generate_equations_included_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2, u_x_1_12, u_y_1_12, u_x_2_12, u_y_2_12)

Matrix([
[r_x_1 - r_x_2],
[r_y_1 - r_y_2]])

In [157]:
u_x_2_23 = L2
u_y_2_23 = 0
u_x_3_23 = 0
u_y_3_23 = 0
generate_equations_included_topology_pr(theta_2, theta_3, r_x_2, r_y_2, r_x_3, r_y_3, u_x_2_23, u_y_2_23, u_x_3_23, u_y_3_23)

Matrix([
[r_x_2 - r_x_3 + 3*cos(theta_2)],
[r_y_2 - r_y_3 + 3*sin(theta_2)]])

In [158]:
u_x_3_34 = L3
u_y_3_34 = 0
u_x_4_34 = 0
u_y_4_34 = 0
generate_equations_included_topology_pr(theta_3, theta_4, r_x_3, r_y_3, r_x_4, r_y_4, u_x_3_34, u_y_3_34, u_x_4_34, u_y_4_34)

Matrix([
[r_x_3 - r_x_4 + 5*cos(theta_3)],
[r_y_3 - r_y_4 + 5*sin(theta_3)]])

In [167]:
u_x_1_14 = Separation
u_y_1_14 = 0
u_x_4_14 = L4
u_y_4_14 = 0
generate_equations_included_topology_pr(theta_1, theta_4, r_x_1, r_y_1, r_x_4, r_y_4, u_x_1_14, u_y_1_14, u_x_4_14, u_y_4_14)

Matrix([
[r_x_1 - r_x_4 + 5*cos(theta_1) - 5*cos(theta_4)],
[r_y_1 - r_y_4 + 5*sin(theta_1) - 5*sin(theta_4)]])

In [168]:
def four_bars_mechanism_example_symbolic(theta_2_value, L2, L3, L4, Separation):

    theta_1, theta_2, theta_3, theta_4 = symbols('theta_1 theta_2 theta_3 theta_4')
    r_x_1, r_y_1, r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_1 r_y_1 r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')


    u_x_1_12 = 0
    u_y_1_12 = 0
    u_x_2_12 = 0
    u_y_2_12 = 0
    restrictions_pr12 = generate_equations_included_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2, u_x_1_12, u_y_1_12, u_x_2_12, u_y_2_12)

    u_x_2_23 = L2
    u_y_2_23 = 0
    u_x_3_23 = 0
    u_y_3_23 = 0
    restrictions_pr23 = generate_equations_included_topology_pr(theta_2, theta_3, r_x_2, r_y_2, r_x_3, r_y_3, u_x_2_23, u_y_2_23, u_x_3_23, u_y_3_23)

    u_x_3_34 = L3
    u_y_3_34 = 0
    u_x_4_34 = 0
    u_y_4_34 = 0
    restrictions_pr34 = generate_equations_included_topology_pr(theta_3, theta_4, r_x_3, r_y_3, r_x_4, r_y_4, u_x_3_34, u_y_3_34, u_x_4_34, u_y_4_34)

    u_x_1_14 = Separation
    u_y_1_14 = 0
    u_x_4_14 = L4
    u_y_4_14 = 0
    restrictions_pr14 = generate_equations_included_topology_pr(theta_1, theta_4, r_x_1, r_y_1, r_x_4, r_y_4, u_x_1_14, u_y_1_14, u_x_4_14, u_y_4_14)

    # Stacking the matrices vertically to form a single large matrix
    restrictions_to_solve = restrictions_pr12.col_join(restrictions_pr23).col_join(restrictions_pr34).col_join(restrictions_pr14)

    # Substituting the known values into the equations
    restrictions_substituted = restrictions_to_solve.subs({
        r_x_1: 0,
        r_y_1: 0,
        theta_1: 0,
        theta_2: theta_2_value
    })

    # Create a list of equations from the matrix
    equation_list = [Eq(restrictions_substituted[i, 0], 0) for i in range(restrictions_substituted.shape[0])]

    #print(equation_list) 

    return(equation_list)

In [169]:
L2 = 5
L3 = 5
L4 = 5
Separation = 5


theta_2 = np.pi/2


four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation)

[Eq(-r_x_2, 0),
 Eq(-r_y_2, 0),
 Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
 Eq(r_y_2 - r_y_3 + 5.0, 0),
 Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0),
 Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0),
 Eq(-r_x_4 - 5*cos(theta_4) + 5, 0),
 Eq(-r_y_4 - 5*sin(theta_4), 0)]

In [170]:
def four_bars_mechanism_example(theta_2_value, initial_guess, L2, L3, L4, Separation):

    theta_1, theta_2, theta_3, theta_4 = symbols('theta_1 theta_2 theta_3 theta_4')
    r_x_1, r_y_1, r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = symbols('r_x_1 r_y_1 r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4')


    u_x_1_12 = 0
    u_y_1_12 = 0
    u_x_2_12 = 0
    u_y_2_12 = 0
    restrictions_pr12 = generate_equations_included_topology_pr(theta_1, theta_2, r_x_1, r_y_1, r_x_2, r_y_2, u_x_1_12, u_y_1_12, u_x_2_12, u_y_2_12)

    u_x_2_23 = L2
    u_y_2_23 = 0
    u_x_3_23 = 0
    u_y_3_23 = 0
    restrictions_pr23 = generate_equations_included_topology_pr(theta_2, theta_3, r_x_2, r_y_2, r_x_3, r_y_3, u_x_2_23, u_y_2_23, u_x_3_23, u_y_3_23)

    u_x_3_34 = L3
    u_y_3_34 = 0
    u_x_4_34 = 0
    u_y_4_34 = 0
    restrictions_pr34 = generate_equations_included_topology_pr(theta_3, theta_4, r_x_3, r_y_3, r_x_4, r_y_4, u_x_3_34, u_y_3_34, u_x_4_34, u_y_4_34)

    u_x_1_14 = Separation
    u_y_1_14 = 0
    u_x_4_14 = L4
    u_y_4_14 = 0
    restrictions_pr14 = generate_equations_included_topology_pr(theta_1, theta_4, r_x_1, r_y_1, r_x_4, r_y_4, u_x_1_14, u_y_1_14, u_x_4_14, u_y_4_14)

    # Stacking the matrices vertically to form a single large matrix
    restrictions_to_solve = restrictions_pr12.col_join(restrictions_pr23).col_join(restrictions_pr34).col_join(restrictions_pr14)

    # Substituting the known values into the equations
    restrictions_substituted = restrictions_to_solve.subs({
        r_x_1: 0,
        r_y_1: 0,
        theta_1: 0,
        theta_2: theta_2_value
    })

    # Create a list of equations from the matrix
    equation_list = [Eq(restrictions_substituted[i, 0], 0) for i in range(restrictions_substituted.shape[0])]

    # This will try to solve the system of equations (you can run it at your discretion)
    solution = solve(equation_list)

    # Print the prepared equations (you can comment this out if not needed)
    for eq in equation_list:
        print(eq)

    # Convert the symbolic equations to lambda functions for fsolve
    #func = lambdify(( r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4), list(restrictions_pr12 + restrictions_pr23 + restrictions_pr34 + restrictions_pr14), "numpy")
    func = lambdify(( r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4), list(equation_list), "numpy") #the ones already with the topology included

    def equations_for_fsolve(vars):
         return func(*vars)

    # Use fsolve to find the solution
    numerical_solution = fsolve(equations_for_fsolve, initial_guess) 
    return(numerical_solution) #r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4


In [171]:
import plotly.graph_objects as go



import plotly.graph_objects as go
import numpy as np

def plot_mechanism_updated(solution):
    # Extracting the coordinates from the solution
    r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4 = solution[0:6]
    
    # Create traces for each line
    trace1 = go.Scatter(x=[r_x_2, r_x_3], y=[r_y_2, r_y_3], mode='lines+markers', name='Link 2-3')
    trace2 = go.Scatter(x=[r_x_3, r_x_4], y=[r_y_3, r_y_4], mode='lines+markers', name='Link 3-4')
    trace3 = go.Scatter(x=[r_x_4, 5], y=[r_y_4, 0], mode='lines+markers', name='Link 4-Base')
    
    # Define layout with equal aspect ratio and specified axis ranges
    layout = go.Layout(title='Four Bars Mechanism Visualization',
                       xaxis=dict(title='X Coordinate', scaleanchor="y", scaleratio=1, range=[-15, 15]),
                       yaxis=dict(title='Y Coordinate', scaleanchor="x", scaleratio=1, range=[-15, 15]),
                       showlegend=True,
                       autosize=False,  # Disable automatic sizing to maintain aspect ratio
                       width=500,  # Set width of the plot
                       height=500)  # Set height of the plot
    
    # Create figure and plot
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()




# Testing the updated plotting function with the sample solution
#plot_mechanism_updated(solution)


In [ ]:
import plotly.graph_objs as go

def plot_mechanism_updated_json(new_input, Separation):
    # Extract values from the input dictionary using string keys
    r_x_2_value = new_input['r_x_2']
    r_y_2_value = new_input['r_y_2']
    r_x_3_value = new_input['r_x_3']
    r_y_3_value = new_input['r_y_3']
    r_x_4_value = new_input['r_x_4']
    r_y_4_value = new_input['r_y_4']

    print(r_x_2_value)
    
    # Create traces for each line
    trace1 = go.Scatter(x=[r_x_2_value, r_x_3_value], y=[r_y_2_value, r_y_3_value], mode='lines+markers', name='Link 2-3')
    trace2 = go.Scatter(x=[r_x_3_value, r_x_4_value], y=[r_y_3_value, r_y_4_value], mode='lines+markers', name='Link 3-4')
    trace3 = go.Scatter(x=[r_x_4_value, Separation], y=[r_y_4_value, 0], mode='lines+markers', name='Link 4-Base')
    
    # Define layout with equal aspect ratio and specified axis ranges
    layout = go.Layout(title='Four Bars Mechanism Visualization',
                       xaxis=dict(title='X Coordinate', scaleanchor="y", scaleratio=1, range=[-15, 15]),
                       yaxis=dict(title='Y Coordinate', scaleanchor="x", scaleratio=1, range=[-15, 15]),
                       showlegend=True,
                       autosize=False,  # Disable automatic sizing to maintain aspect ratio
                       width=500,  # Set width of the plot
                       height=500)  # Set height of the plot
    
    # Create figure and plot
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()

test_results2 = {
    'r_x_2': 0.0,
    'r_x_3': 1.22464679914735e-16,
    'r_y_2': 0.0,
    'r_y_3': 2.00000000000000,
    'r_x_4': 3.06161699786838e-16,
    'r_y_4': 5.00000000000000
}

plot_mechanism_updated_json(test_results2,Separation)
#plot_mechanism_updated_json(new_dict,Separation)


In [172]:
# Testing that it solves it properly

L2 = 5
L3 = 5
L4 = 5
Separation = 5

initial_guess = [ 0, 0, 0, 5, 5, 5, -np.pi/2, 0]  #The guess in this order: r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4

theta_2 = np.pi/2

solution = four_bars_mechanism_example(theta_2, initial_guess, L2, L3, L4, Separation)
print(solution)

Eq(-r_x_2, 0)
Eq(-r_y_2, 0)
Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0)
Eq(r_y_2 - r_y_3 + 5.0, 0)
Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0)
Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0)
Eq(-r_x_4 - 5*cos(theta_4) + 5, 0)
Eq(-r_y_4 - 5*sin(theta_4), 0)
[ 1.49011612e-08  1.49011612e-08  0.00000000e+00  5.00000000e+00
  5.00000000e+00  5.00000000e+00 -1.57079633e+00  0.00000000e+00]


In [174]:
#and here we have those equations:

sym_equations_to_solve = four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation)

sym_equations_to_solve



[Eq(-r_x_2, 0),
 Eq(-r_y_2, 0),
 Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
 Eq(r_y_2 - r_y_3 + 5.0, 0),
 Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0),
 Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0),
 Eq(-r_x_4 - 5*cos(theta_4) + 5, 0),
 Eq(-r_y_4 - 5*sin(theta_4), 0)]

In [175]:
import numpy as np
from scipy.optimize import fsolve
import sympy as sp

def solve_numerically(eqs, initial_guess):
    # Convert the Eq objects into expressions that evaluate to zero
    expressions = [eq.lhs - eq.rhs for eq in eqs]

    # Extract all the symbols from the expressions
    symbols = list(set().union(*[expr.free_symbols for expr in expressions]))

    # Lambdify the expressions
    func = sp.lambdify(symbols, expressions, "numpy")

    # Define a wrapper function for fsolve
    def wrapper(params):
        return func(*params)

    # Solve the equations numerically
    solution = fsolve(wrapper, initial_guess)

    return dict(zip(symbols, solution))

# Your input equations are stored in sym_equations_to_solve
solution = solve_numerically(sym_equations_to_solve, initial_guess)
print(solution)


{r_y_3: 5.0, r_x_3: 3.0616172175744623e-16, r_y_4: 7.185155911688072e-10, theta_4: 12.566370614221553, r_x_2: 3.1161407054241035e-23, theta_3: -1.5707963266480782, r_y_2: -1.9207247182445632e-21, r_x_4: 1.069627661854941e-09}


In [176]:
(type(sym_equations_to_solve))

list

In [20]:

print(type(sym_equations_to_solve[0]))

<class 'sympy.core.relational.Equality'>


In [177]:
solution = four_bars_mechanism_example(theta_2, initial_guess, L2, L3, L4, Separation)


plot_mechanism_updated(solution)

Eq(-r_x_2, 0)
Eq(-r_y_2, 0)
Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0)
Eq(r_y_2 - r_y_3 + 5.0, 0)
Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0)
Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0)
Eq(-r_x_4 - 5*cos(theta_4) + 5, 0)
Eq(-r_y_4 - 5*sin(theta_4), 0)


In [178]:
# Testing that it solves it properly

L2 = 2
L3 = 3
L4 = 4
Separation = 3

initial_guess = [ 0, 0, 0, 7, 6, 6, 0, -np.pi/2]  #The guess in this order: r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4

theta_2 = np.pi/2

solution = four_bars_mechanism_example(theta_2, initial_guess, L2, L3, L4, Separation)
print(solution)

Eq(-r_x_2, 0)
Eq(-r_y_2, 0)
Eq(r_x_2 - r_x_3 + 1.22464679914735e-16, 0)
Eq(r_y_2 - r_y_3 + 2.0, 0)
Eq(r_x_3 - r_x_4 + 3*cos(theta_3), 0)
Eq(r_y_3 - r_y_4 + 3*sin(theta_3), 0)
Eq(-r_x_4 - 4*cos(theta_4) + 3, 0)
Eq(-r_y_4 - 4*sin(theta_4), 0)
[ 1.49011612e-08  1.49011612e-08  0.00000000e+00  7.00000000e+00
  6.00000000e+00  6.00000000e+00  0.00000000e+00 -1.57079633e+00]


In [179]:
plot_mechanism_updated(solution)

In [181]:
# # Modify the equations_evaluator to unpack the variables
# def equations_evaluator_unpack(vars):
#     r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = vars
#     return equations_evaluator(r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4)

# # Modify the four_bars_mechanism function to use the unpacking evaluator
# def four_bars_mechanism_unpack_eval(theta_2_value, initial_guess, L2, L3, L4, Separation):
#     # Use fsolve with the unpacking evaluator function
#     numerical_solution = fsolve(equations_evaluator_unpack, initial_guess, xtol=1e-10, maxfev=10000)
    
#     # Adjusting the angles to lie within [0, 2*pi]
#     numerical_solution[6] = numerical_solution[6] % (2 * np.pi)
#     numerical_solution[7] = numerical_solution[7] % (2 * np.pi)
    
#     return numerical_solution

# # Testing the modified function with unpacking evaluator again
# solution_unpack_final = four_bars_mechanism_unpack_eval(theta_2, initial_guess, L2, L3, L4, Separation)
# solution_unpack_final


In [37]:
plot_mechanism_updated(solution_unpack_final)

In [28]:
# from sympy import Eq, cos, sin


# # Define the symbolic variables again
# r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4 theta_3 theta_4')

# # Sample equations provided
# sample_equations = [
#     Eq(-r_x_2, 0),
#     Eq(-r_y_2, 0),
#     Eq(r_x_2 - r_x_3 + 3.67394039744206e-16, 0),
#     Eq(r_y_2 - r_y_3 + 6.0, 0),
#     Eq(r_x_3 - r_x_4 + 6*cos(theta_3), 0),
#     Eq(r_y_3 - r_y_4 + 6*sin(theta_3), 0),
#     Eq(6 - 6*cos(theta_4), 0),
#     Eq(-6*sin(theta_4), 0)
# ]

# def equations_evaluator_from_symbolic(vars, equations):
#     # Unpack the variables
#     r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = vars
    
#     # Evaluate each equation and store its residual
#     values = []
#     for eq in equations:
#         # Lambdify the equation to get a function
#         func = lambdify((r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4), eq.lhs - eq.rhs, "numpy")
#         # Evaluate the function and append to the values list
#         values.append(func(r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4))
    
#     return values

# # Test the evaluator with the sample equations and a set of test values
# test_values = [0, 0, 0, 6, -5.3459, 3.2757, 3.6129, 0]
# evaluated_equations = equations_evaluator_from_symbolic(test_values, sample_equations)
# evaluated_equations


In [29]:
# # Lambdify all the equations once
# lambdified_functions = [lambdify((r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4), eq.lhs - eq.rhs, "numpy") for eq in sample_equations]

# def equations_evaluator_from_symbolic(vars):
#     # Unpack the variables
#     r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = vars
    
#     # Evaluate each lambdified function and store its residual
#     values = [func(r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4) for func in lambdified_functions]
    
#     return values

# # Test the evaluator with the sample equations and a set of test values
# evaluated_equations_corrected = equations_evaluator_from_symbolic(test_values)
# evaluated_equations_corrected


## Another Way

In [183]:
from scipy.optimize import fsolve

# Define the symbolic equations
given_equations = [
    Eq(-r_x_2, 0),
    Eq(-r_y_2, 0),
    Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
    Eq(r_y_2 - r_y_3 + 5.0, 0),
    Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0),
    Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0),
    Eq(-r_x_4 - 4*cos(theta_4) + 3, 0),
    Eq(-r_y_4 - 4*sin(theta_4), 0)
]


# Lambdify all the equations
lambdified_eqs = [lambdify((r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4), eq.lhs - eq.rhs, "numpy") for eq in given_equations]

def solve_equations(equations, initial_guess):
    # Evaluator function for fsolve
    def equations_evaluator(vars):
        return [eq(*vars) for eq in equations]

    # Solve the system of equations using fsolve
    solution = fsolve(equations_evaluator, initial_guess, xtol=1e-10, maxfev=10000)

    # Adjusting the angles to lie within [0, 2*pi]
    solution[6] = solution[6] % (2 * np.pi)
    solution[7] = solution[7] % (2 * np.pi)
    
    return solution

# Given initial guess
initial_guess = [0, 0, 0, 5, -4.5, 3, 3.5, 0]

# Solve using the function
solution_final = solve_equations(lambdified_eqs, initial_guess)
solution_final


array([ 1.52352948e-27, -3.05705078e-29,  3.06161700e-16,  5.00000000e+00,
       -9.98730686e-01,  1.00761588e-01,  4.51129015e+00,  6.25799225e+00])

In [185]:
plot_mechanism_updated(solution_final)

## Pure symbolic


In [202]:
import sympy as sp

# Define the symbols
r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = sp.symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4 theta_3 theta_4')

# Define the equations
# equations = [
#     sp.Eq(-r_x_2, 0),
#     sp.Eq(-r_y_2, 0),
#     sp.Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
#     sp.Eq(r_y_2 - r_y_3 + 5.0, 0),
#     sp.Eq(r_x_3 - r_x_4 + 5*sp.cos(theta_3), 0),
#     sp.Eq(r_y_3 - r_y_4 + 5*sp.sin(theta_3), 0),
#     sp.Eq(5 - 5*sp.cos(theta_4), 0),
#     sp.Eq(-5*sp.sin(theta_4), 0)
# ]

#and here we have those equations:

L2 = 5
L3 = 5
L4 = 5
Separation = 5

#initial_guess = [ 0, 0, 0, 5, 5, 5, -np.pi/2, 0]  #The guess in this order: r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4

theta_2 = np.pi/2

sym_equations_to_solve = four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation)

#sym_equations_to_solve



# Solve the equations
solutions = sp.solve(sym_equations_to_solve)

solutions


[{r_x_2: 0.0,
  r_x_3: 3.06161699786838e-16,
  r_y_2: 0.0,
  r_y_3: 5.00000000000000,
  r_y_4: 9.37349864163659e-33,
  r_x_4: 8.78624767847630e-66,
  theta_3: 4.71238898038469,
  theta_4: -1.87469972832732e-33},
 {r_x_2: 0.0,
  r_x_3: 3.06161699786838e-16,
  r_y_2: 0.0,
  r_y_3: 5.00000000000000,
  r_y_4: 5.00000000000000,
  r_x_4: 5.00000000000000,
  theta_3: 6.28318530717959,
  theta_4: -1.57079632679490}]

In [192]:
type(solutions)

list

In [204]:
solutions[0]


{r_x_2: 0.0,
 r_x_3: 3.06161699786838e-16,
 r_y_2: 0.0,
 r_y_3: 5.00000000000000,
 r_y_4: 9.37349864163659e-33,
 r_x_4: 8.78624767847630e-66,
 theta_3: 4.71238898038469,
 theta_4: -1.87469972832732e-33}

In [205]:

solutions_theta_3_1 = solutions[0]

In [206]:
solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

solution_json

{'r_x_2': 0.0,
 'r_x_3': 3.06161699786838e-16,
 'r_y_2': 0.0,
 'r_y_3': 5.00000000000000,
 'r_y_4': 9.37349864163659e-33,
 'r_x_4': 8.78624767847630e-66,
 'theta_3': 4.71238898038469,
 'theta_4': -1.87469972832732e-33}

In [201]:


plot_mechanism_updated_json({'r_x_2': 0.0,
 'r_x_3': 1.83697019872103e-16,
 'r_y_2': 0.0,
 'r_y_3': 3.00000000000000,
 'r_y_4': -1.94809625674159,
 'r_x_4': 0.718570409925080,
 'theta_3': 4.85660242306916,
 'theta_4': 0.706749687228324},Separation)

0.0


In [194]:
# Extract the expression for r_x_4 in terms of theta_3
#r_x_4_expr = solutions[r_x_4]
r_x_4_expr = solutions[0][r_x_4]

# Solve for the value of theta_3 that minimizes the absolute value of r_x_4
theta_3_solution = sp.solve(sp.Eq(r_x_4_expr, 0), theta_3, domain=sp.Interval(0, 2*sp.pi))

theta_3_solution


[]

In [195]:
# Substitute the values of theta_3 into the solutions
solutions_theta_3_1 = {key: val.subs(theta_3, theta_3_solution[0]) for key, val in solutions.items()}
#solutions_theta_3_2 = {key: val.subs(theta_3, theta_3_solution[1]) for key, val in solutions.items()}

#solutions_theta_3_1, solutions_theta_3_2
solutions_theta_3_1


AttributeError: 'list' object has no attribute 'items'

### Another Topology V1

In [189]:
import sympy as sp

# Define the symbols
r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = sp.symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4 theta_3 theta_4')


#and here we have those equations:

L2 = 3
L3 = 5
L4 = 3
Separation = 3


theta_2 = np.pi/2

sym_equations_to_solve = four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation)

print(sym_equations_to_solve)






[Eq(-r_x_2, 0), Eq(-r_y_2, 0), Eq(r_x_2 - r_x_3 + 1.83697019872103e-16, 0), Eq(r_y_2 - r_y_3 + 3.0, 0), Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0), Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0), Eq(-r_x_4 - 3*cos(theta_4) + 3, 0), Eq(-r_y_4 - 3*sin(theta_4), 0)]


In [190]:

# Solve the equations
solutions = sp.solve(sym_equations_to_solve)

solutions


[{r_x_2: 0.0,
  r_x_3: 1.83697019872103e-16,
  r_y_2: 0.0,
  r_y_3: 3.00000000000000,
  r_y_4: -1.94809625674159,
  r_x_4: 0.718570409925080,
  theta_3: 4.85660242306916,
  theta_4: 0.706749687228324},
 {r_x_2: 0.0,
  r_x_3: 1.83697019872103e-16,
  r_y_2: 0.0,
  r_y_3: 3.00000000000000,
  r_y_4: 2.28142959007492,
  r_x_4: 4.94809625674159,
  theta_3: 6.13897186449512,
  theta_4: -2.27754601402322}]

In [191]:
# Extract the expression for r_x_4 in terms of theta_3
r_x_4_expr = solutions[r_x_4]

# Solve for the value of theta_3 that minimizes the absolute value of r_x_4
theta_3_solution = sp.solve(sp.Eq(r_x_4_expr, 0), theta_3, domain=sp.Interval(0, 2*sp.pi))

theta_3_solution


TypeError: list indices must be integers or slices, not Symbol

In [141]:
# Substitute the values of theta_3 into the solutions
solutions_theta_3_1 = {key: val.subs(theta_3, theta_3_solution[0]) for key, val in solutions.items()}
#solutions_theta_3_2 = {key: val.subs(theta_3, theta_3_solution[1]) for key, val in solutions.items()}

#solutions_theta_3_1, solutions_theta_3_2
solutions_theta_3_1


{r_x_2: 0.0,
 r_x_3: 1.83697019872103e-16,
 r_y_2: 0.0,
 r_y_3: 3.00000000000000,
 r_x_4: 4.89858719658941e-16,
 r_y_4: 8.00000000000000,
 theta_4: 0.0}

In [146]:
solution_json = {str(key): value for key, value in solutions_theta_3_1.items()}

solution_json

{'r_x_2': 0.0,
 'r_x_3': 1.83697019872103e-16,
 'r_y_2': 0.0,
 'r_y_3': 3.00000000000000,
 'r_x_4': 4.89858719658941e-16,
 'r_y_4': 8.00000000000000,
 'theta_4': 0.0}

In [147]:


plot_mechanism_updated_json({'r_x_2': 0.0,
 'r_x_3': 1.83697019872103e-16,
 'r_y_2': 0.0,
 'r_y_3': 3.00000000000000,
 'r_x_4': 4.89858719658941e-16,
 'r_y_4': 8.00000000000000,
 'theta_4': 0.0},Separation)

0.0


### Another Topology V2

In [132]:
import sympy as sp

# Define the symbols
r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = sp.symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4 theta_3 theta_4')

# Define the equations
# equations = [
#     sp.Eq(-r_x_2, 0),
#     sp.Eq(-r_y_2, 0),
#     sp.Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
#     sp.Eq(r_y_2 - r_y_3 + 5.0, 0),
#     sp.Eq(r_x_3 - r_x_4 + 5*sp.cos(theta_3), 0),
#     sp.Eq(r_y_3 - r_y_4 + 5*sp.sin(theta_3), 0),
#     sp.Eq(5 - 5*sp.cos(theta_4), 0),
#     sp.Eq(-5*sp.sin(theta_4), 0)
# ]

#and here we have those equations:

L2 = 3
L3 = 5
L4 = 3
Separation = 3

#initial_guess = [ 0, 0, 0, 5, 5, 5, -np.pi/2, 0]  #The guess in this order: r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4

theta_2 = np.pi/2

sym_equations_to_solve = four_bars_mechanism_example_symbolic(theta_2, L2, L3, L4, Separation)

print(sym_equations_to_solve)






[Eq(-r_x_2, 0), Eq(-r_y_2, 0), Eq(r_x_2 - r_x_3 + 1.83697019872103e-16, 0), Eq(r_y_2 - r_y_3 + 3.0, 0), Eq(r_x_3 - r_x_4 + 5*cos(theta_3), 0), Eq(r_y_3 - r_y_4 + 5*sin(theta_3), 0), Eq(3 - 3*cos(theta_4), 0), Eq(-3*sin(theta_4), 0)]


In [133]:
# Solve the equations
solutions = sp.solve(sym_equations_to_solve)

solutions

{r_x_2: 0.0,
 r_x_3: 1.83697019872103e-16,
 r_y_2: 0.0,
 r_y_3: 3.00000000000000,
 r_x_4: 5.0*cos(theta_3) + 1.83697019872103e-16,
 r_y_4: 5.0*sin(theta_3) + 3.0,
 theta_4: 0.0}

In [134]:
### NUMERICAL AGAIN

# Use a numerical solver to find values for theta_4 within [0, 2*pi]
# theta_4_numerical_solutions = sp.nsolve(solutions[-2:], theta_4, (0, 2*sp.pi))

# theta_4_numerical_solutions


def solve_system(equations):
    # Define the symbols
    r_x_2, r_y_2, r_x_3, r_y_3, r_x_4, r_y_4, theta_3, theta_4 = sp.symbols('r_x_2 r_y_2 r_x_3 r_y_3 r_x_4 r_y_4 theta_3 theta_4')

    # Attempt direct symbolic solution
    solutions = sp.solve(equations)
    if solutions:
        return solutions

    # Numerically solve for theta_4
    theta_4_solution_cos = sp.nsolve(equations[-2], theta_4, 1)
    theta_4_solution_sin = sp.nsolve(equations[-1], theta_4, 1)

    # Use the theta_4 value that's closer to 0 (more straightforward)
    theta_4_value = min(theta_4_solution_cos, theta_4_solution_sin, key=abs)

    # Substitute the value of theta_4 into the equations
    substituted_equations = [eq.subs(theta_4, theta_4_value) for eq in equations[:-2]]

    # Solve the substituted equations
    solutions_substituted = sp.solve(substituted_equations)

    # Numerically solve for theta_3 that makes dependent variables close to desired values
    r_x_4_expr = solutions_substituted[0][r_x_4]
    theta_3_solution = sp.nsolve(sp.Eq(r_x_4_expr, 0), theta_3, 1)

    # Substitute the value of theta_3 into the solutions
    complete_solutions = {key: val.subs(theta_3, theta_3_solution) for key, val in solutions_substituted[0].items()}
    
    return complete_solutions

# Test the function with the provided equations
# test_equations = [
#     sp.Eq(-r_x_2, 0),
#     sp.Eq(-r_y_2, 0),
#     sp.Eq(r_x_2 - r_x_3 + 3.06161699786838e-16, 0),
#     sp.Eq(r_y_2 - r_y_3 + 5.0, 0),
#     sp.Eq(r_x_3 - r_x_4 + 5*sp.cos(theta_3), 0),
#     sp.Eq(r_y_3 - r_y_4 + 5*sp.sin(theta_3), 0),
#     sp.Eq(5 - 6*sp.cos(theta_4), 0),
#     sp.Eq(-6*sp.sin(theta_4), 0)
# ]

test_results = solve_system(sym_equations_to_solve)
test_results


{r_x_2: 0.0,
 r_x_3: 1.83697019872103e-16,
 r_y_2: 0.0,
 r_y_3: 3.00000000000000,
 r_x_4: 5.0*cos(theta_3) + 1.83697019872103e-16,
 r_y_4: 5.0*sin(theta_3) + 3.0,
 theta_4: 0.0}

In [135]:
type(test_results)

dict

In [125]:
# test_results = {
#     r_x_2: 0.0,
#     r_x_3: 1.22464679914735e-16,
#     r_y_2: 0.0,
#     r_y_3: 2.00000000000000,
#     r_x_4: 3.06161699786838e-16,
#     r_y_4: 5.00000000000000
# }

new_dict_result = {str(key): value for key, value in test_results.items()}

print(new_dict_result)


{'r_x_2': 0.0, 'r_x_3': 1.22464679914735e-16, 'r_y_2': 0.0, 'r_y_3': 2.00000000000000, 'r_x_4': 3.06161699786838e-16, 'r_y_4': 5.00000000000000}


In [113]:
# {
#     r_x_2: 0.0,
#     r_x_3: 1.22464679914735e-16,
#     r_y_2: 0.0,
#     r_y_3: 2.00000000000000,
#     r_x_4: 3.06161699786838e-16,
#     r_y_4: 5.00000000000000
# }

In [136]:
import plotly.graph_objs as go

def plot_mechanism_updated_json(new_input, Separation):
    # Extract values from the input dictionary using string keys
    r_x_2_value = new_input['r_x_2']
    r_y_2_value = new_input['r_y_2']
    r_x_3_value = new_input['r_x_3']
    r_y_3_value = new_input['r_y_3']
    r_x_4_value = new_input['r_x_4']
    r_y_4_value = new_input['r_y_4']

    print(r_x_2_value)
    
    # Create traces for each line
    trace1 = go.Scatter(x=[r_x_2_value, r_x_3_value], y=[r_y_2_value, r_y_3_value], mode='lines+markers', name='Link 2-3')
    trace2 = go.Scatter(x=[r_x_3_value, r_x_4_value], y=[r_y_3_value, r_y_4_value], mode='lines+markers', name='Link 3-4')
    trace3 = go.Scatter(x=[r_x_4_value, Separation], y=[r_y_4_value, 0], mode='lines+markers', name='Link 4-Base')
    
    # Define layout with equal aspect ratio and specified axis ranges
    layout = go.Layout(title='Four Bars Mechanism Visualization',
                       xaxis=dict(title='X Coordinate', scaleanchor="y", scaleratio=1, range=[-15, 15]),
                       yaxis=dict(title='Y Coordinate', scaleanchor="x", scaleratio=1, range=[-15, 15]),
                       showlegend=True,
                       autosize=False,  # Disable automatic sizing to maintain aspect ratio
                       width=500,  # Set width of the plot
                       height=500)  # Set height of the plot
    
    # Create figure and plot
    fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig.show()

test_results2 = {
    'r_x_2': 0.0,
    'r_x_3': 1.22464679914735e-16,
    'r_y_2': 0.0,
    'r_y_3': 2.00000000000000,
    'r_x_4': 3.06161699786838e-16,
    'r_y_4': 5.00000000000000
}

plot_mechanism_updated_json(test_results2,Separation)
#plot_mechanism_updated_json(new_dict,Separation)


0.0


In [128]:
new_dict_result

{'r_x_2': 0.0,
 'r_x_3': 1.22464679914735e-16,
 'r_y_2': 0.0,
 'r_y_3': 2.00000000000000,
 'r_x_4': 3.06161699786838e-16,
 'r_y_4': 5.00000000000000}

In [143]:


plot_mechanism_updated_json({'r_x_2': 0.0,
 'r_x_3': 1.22464679914735e-16,
 'r_y_2': 0.0,
 'r_y_3': 2.00000000000000,
 'r_x_4': 3.06161699786838e-16,
 'r_y_4': 5.00000000000000},Separation)

0.0
